<a href="https://colab.research.google.com/github/vitoropdm-8/dotfiles/blob/master/Credit_Card.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
#Imports
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
import seaborn as sns
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [5]:
#Instanciando as bases
previous=pd.read_csv('/content/drive/MyDrive/Bases Project LW/previous_application.csv')
credit_card=pd.read_csv('/content/drive/MyDrive/Bases Project LW/credit_card_balance.csv')

In [6]:
# Calcule a proporção de valores nulos em cada coluna
null_ratios = credit_card.isnull().sum() / credit_card.shape[0]

# Configurar a opção de exibição para evitar notação exponencial
pd.set_option('display.float_format', '{:.2f}'.format)

# Criar um novo DataFrame a partir dos resultados
result_df = pd.DataFrame(null_ratios, columns=['Null Ratio'])

# Ordenar o DataFrame com base nos valores das colunas
result_df.sort_values(by='Null Ratio', ascending=False)

,Null Ratio
AMT_PAYMENT_CURRENT,0.20
AMT_DRAWINGS_ATM_CURRENT,0.20
CNT_DRAWINGS_POS_CURRENT,0.20
AMT_DRAWINGS_OTHER_CURRENT,0.20
AMT_DRAWINGS_POS_CURRENT,0.20
CNT_DRAWINGS_OTHER_CURRENT,0.20
CNT_DRAWINGS_ATM_CURRENT,0.20
CNT_INSTALMENT_MATURE_CUM,0.08
AMT_INST_MIN_REGULARITY,0.08
SK_ID_PREV,0.00


In [12]:
# Feature Add - NUMBER OF LOANS PER CUSTOMER
import gc
CCB = credit_card[0:]
grp = CCB.groupby(by = ['SK_ID_CURR'])['SK_ID_PREV'].nunique().reset_index().rename(index = str, columns = {'SK_ID_PREV': 'NO_LOANS'})
CCB = CCB.merge(grp, on = ['SK_ID_CURR'], how = 'left')
del grp
gc.collect()
print(CCB.dtypes, CCB.shape)

SK_ID_PREV                      int64
SK_ID_CURR                      int64
MONTHS_BALANCE                  int64
AMT_BALANCE                   float64
AMT_CREDIT_LIMIT_ACTUAL         int64
AMT_DRAWINGS_ATM_CURRENT      float64
AMT_DRAWINGS_CURRENT          float64
AMT_DRAWINGS_OTHER_CURRENT    float64
AMT_DRAWINGS_POS_CURRENT      float64
AMT_INST_MIN_REGULARITY       float64
AMT_PAYMENT_CURRENT           float64
AMT_PAYMENT_TOTAL_CURRENT     float64
AMT_RECEIVABLE_PRINCIPAL      float64
AMT_RECIVABLE                 float64
AMT_TOTAL_RECEIVABLE          float64
CNT_DRAWINGS_ATM_CURRENT      float64
CNT_DRAWINGS_CURRENT            int64
CNT_DRAWINGS_OTHER_CURRENT    float64
CNT_DRAWINGS_POS_CURRENT      float64
CNT_INSTALMENT_MATURE_CUM     float64
NAME_CONTRACT_STATUS           object
SK_DPD                          int64
SK_DPD_DEF                      int64
NO_LOANS                        int64
dtype: object (3840312, 24)


In [13]:
# Feature Add - Number of times Days Past Due occurred

def f(DPD):

    # DPD is a series of values of SK_DPD for each of the groupby combination
    # We convert it to a list to get the number of SK_DPD values NOT EQUALS ZERO
    x = DPD.tolist()
    c = 0
    for i,j in enumerate(x):
        if j != 0:
            c += 1

    return c

grp = CCB.groupby(by = ['SK_ID_CURR', 'SK_ID_PREV']).apply(lambda x: f(x.SK_DPD)).reset_index().rename(index = str, columns = {0: 'NO_DPD'})
grp1 = grp.groupby(by = ['SK_ID_CURR'])['NO_DPD'].mean().reset_index().rename(index = str, columns = {'NO_DPD' : 'DPD_COUNT'})

CCB = CCB.merge(grp1, on = ['SK_ID_CURR'], how = 'left')
del grp1
del grp
gc.collect()

print(CCB.dtypes, CCB.shape)

SK_ID_PREV                      int64
SK_ID_CURR                      int64
MONTHS_BALANCE                  int64
AMT_BALANCE                   float64
AMT_CREDIT_LIMIT_ACTUAL         int64
AMT_DRAWINGS_ATM_CURRENT      float64
AMT_DRAWINGS_CURRENT          float64
AMT_DRAWINGS_OTHER_CURRENT    float64
AMT_DRAWINGS_POS_CURRENT      float64
AMT_INST_MIN_REGULARITY       float64
AMT_PAYMENT_CURRENT           float64
AMT_PAYMENT_TOTAL_CURRENT     float64
AMT_RECEIVABLE_PRINCIPAL      float64
AMT_RECIVABLE                 float64
AMT_TOTAL_RECEIVABLE          float64
CNT_DRAWINGS_ATM_CURRENT      float64
CNT_DRAWINGS_CURRENT            int64
CNT_DRAWINGS_OTHER_CURRENT    float64
CNT_DRAWINGS_POS_CURRENT      float64
CNT_INSTALMENT_MATURE_CUM     float64
NAME_CONTRACT_STATUS           object
SK_DPD                          int64
SK_DPD_DEF                      int64
NO_LOANS                        int64
DPD_COUNT                     float64
dtype: object (3840312, 25)


In [14]:
CCB.head(20)

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,AMT_PAYMENT_CURRENT,AMT_PAYMENT_TOTAL_CURRENT,AMT_RECEIVABLE_PRINCIPAL,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_DRAWINGS_OTHER_CURRENT,CNT_DRAWINGS_POS_CURRENT,CNT_INSTALMENT_MATURE_CUM,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF,NO_LOANS,DPD_COUNT
0,2562384,378907,-6,56.97,135000,0.00,877.50,0.00,877.50,1700.33,1800.00,1800.00,0.00,0.00,0.00,0.00,1,0.00,1.00,35.00,Active,0,0,1,1.00
1,2582071,363914,-1,63975.56,45000,2250.00,2250.00,0.00,0.00,2250.00,2250.00,2250.00,60175.08,64875.56,64875.56,1.00,1,0.00,0.00,69.00,Active,0,0,1,1.00
2,1740877,371185,-7,31815.22,450000,0.00,0.00,0.00,0.00,2250.00,2250.00,2250.00,26926.42,31460.08,31460.08,0.00,0,0.00,0.00,30.00,Active,0,0,1,0.00
3,1389973,337855,-4,236572.11,225000,2250.00,2250.00,0.00,0.00,11795.76,11925.00,11925.00,224949.29,233048.97,233048.97,1.00,1,0.00,0.00,10.00,Active,0,0,1,0.00
4,1891521,126868,-1,453919.46,450000,0.00,11547.00,0.00,11547.00,22924.89,27000.00,27000.00,443044.40,453919.46,453919.46,0.00,1,0.00,1.00,101.00,Active,0,0,1,1.00
5,2646502,380010,-7,82903.82,270000,0.00,0.00,0.00,0.00,4449.10,3825.00,3825.00,80519.04,82773.32,82773.32,0.00,0,0.00,0.00,2.00,Active,7,0,1,1.00
6,1079071,171320,-6,353451.65,585000,67500.00,67500.00,0.00,0.00,14684.17,15750.00,15750.00,345433.86,351881.15,351881.15,1.00,1,0.00,0.00,6.00,Active,0,0,1,0.00
7,2095912,118650,-7,47962.12,45000,45000.00,45000.00,0.00,0.00,0.00,264.69,0.00,44735.31,47962.12,47962.12,1.00,1,0.00,0.00,51.00,Active,0,0,1,2.00
8,2181852,367360,-4,291543.08,292500,90000.00,289339.42,0.00,199339.42,130.50,4093.51,4093.51,285376.41,286831.58,286831.58,3.00,8,0.00,5.00,3.00,Active,0,0,1,0.00
9,1235299,203885,-5,201261.20,225000,76500.00,111026.70,0.00,34526.70,6338.34,45000.00,45000.00,192793.27,197224.70,197224.70,3.00,9,0.00,6.00,38.00,Active,0,0,1,5.00


In [15]:
#Feature Add - AVERAGE OF DAYS PAST DUE PER CUSTOMER
grp = CCB.groupby(by= ['SK_ID_CURR'])['SK_DPD'].mean().reset_index().rename(index = str, columns = {'SK_DPD': 'AVG_DPD'})
CCB = CCB.merge(grp, on = ['SK_ID_CURR'], how = 'left')
del grp
gc.collect()

print(CCB.dtypes, CCB.shape)

SK_ID_PREV                      int64
SK_ID_CURR                      int64
MONTHS_BALANCE                  int64
AMT_BALANCE                   float64
AMT_CREDIT_LIMIT_ACTUAL         int64
AMT_DRAWINGS_ATM_CURRENT      float64
AMT_DRAWINGS_CURRENT          float64
AMT_DRAWINGS_OTHER_CURRENT    float64
AMT_DRAWINGS_POS_CURRENT      float64
AMT_INST_MIN_REGULARITY       float64
AMT_PAYMENT_CURRENT           float64
AMT_PAYMENT_TOTAL_CURRENT     float64
AMT_RECEIVABLE_PRINCIPAL      float64
AMT_RECIVABLE                 float64
AMT_TOTAL_RECEIVABLE          float64
CNT_DRAWINGS_ATM_CURRENT      float64
CNT_DRAWINGS_CURRENT            int64
CNT_DRAWINGS_OTHER_CURRENT    float64
CNT_DRAWINGS_POS_CURRENT      float64
CNT_INSTALMENT_MATURE_CUM     float64
NAME_CONTRACT_STATUS           object
SK_DPD                          int64
SK_DPD_DEF                      int64
NO_LOANS                        int64
DPD_COUNT                     float64
AVG_DPD                       float64
dtype: objec

In [16]:
#Feature Add - % of MINIMUM PAYMENTS MISSED
def f(min_pay, total_pay):

    M = min_pay.tolist()
    T = total_pay.tolist()
    P = len(M)
    c = 0
    # Find the count of transactions when Payment made is less than Minimum Payment
    for i in range(len(M)):
        if T[i] < M[i]:
            c += 1
    return (100*c)/P

grp = CCB.groupby(by = ['SK_ID_CURR']).apply(lambda x: f(x.AMT_INST_MIN_REGULARITY, x.AMT_PAYMENT_CURRENT)).reset_index().rename(index = str, columns = { 0 : 'PERCENTAGE_MISSED_PAYMENTS'})
CCB = CCB.merge(grp, on = ['SK_ID_CURR'], how = 'left')
del grp
gc.collect()

print(CCB.dtypes, CCB.shape)

SK_ID_PREV                      int64
SK_ID_CURR                      int64
MONTHS_BALANCE                  int64
AMT_BALANCE                   float64
AMT_CREDIT_LIMIT_ACTUAL         int64
AMT_DRAWINGS_ATM_CURRENT      float64
AMT_DRAWINGS_CURRENT          float64
AMT_DRAWINGS_OTHER_CURRENT    float64
AMT_DRAWINGS_POS_CURRENT      float64
AMT_INST_MIN_REGULARITY       float64
AMT_PAYMENT_CURRENT           float64
AMT_PAYMENT_TOTAL_CURRENT     float64
AMT_RECEIVABLE_PRINCIPAL      float64
AMT_RECIVABLE                 float64
AMT_TOTAL_RECEIVABLE          float64
CNT_DRAWINGS_ATM_CURRENT      float64
CNT_DRAWINGS_CURRENT            int64
CNT_DRAWINGS_OTHER_CURRENT    float64
CNT_DRAWINGS_POS_CURRENT      float64
CNT_INSTALMENT_MATURE_CUM     float64
NAME_CONTRACT_STATUS           object
SK_DPD                          int64
SK_DPD_DEF                      int64
NO_LOANS                        int64
DPD_COUNT                     float64
AVG_DPD                       float64
PERCENTAGE_M

In [11]:
CCB.head(20)

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,AMT_PAYMENT_CURRENT,AMT_PAYMENT_TOTAL_CURRENT,AMT_RECEIVABLE_PRINCIPAL,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_DRAWINGS_OTHER_CURRENT,CNT_DRAWINGS_POS_CURRENT,CNT_INSTALMENT_MATURE_CUM,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF,PERCENTAGE_MISSED_PAYMENTS
0,2562384,378907,-6,56.97,135000,0.00,877.50,0.00,877.50,1700.33,1800.00,1800.00,0.00,0.00,0.00,0.00,1,0.00,1.00,35.00,Active,0,0,2.13
1,2582071,363914,-1,63975.56,45000,2250.00,2250.00,0.00,0.00,2250.00,2250.00,2250.00,60175.08,64875.56,64875.56,1.00,1,0.00,0.00,69.00,Active,0,0,1.04
2,1740877,371185,-7,31815.22,450000,0.00,0.00,0.00,0.00,2250.00,2250.00,2250.00,26926.42,31460.08,31460.08,0.00,0,0.00,0.00,30.00,Active,0,0,0.00
3,1389973,337855,-4,236572.11,225000,2250.00,2250.00,0.00,0.00,11795.76,11925.00,11925.00,224949.29,233048.97,233048.97,1.00,1,0.00,0.00,10.00,Active,0,0,6.67
4,1891521,126868,-1,453919.46,450000,0.00,11547.00,0.00,11547.00,22924.89,27000.00,27000.00,443044.40,453919.46,453919.46,0.00,1,0.00,1.00,101.00,Active,0,0,1.04
5,2646502,380010,-7,82903.82,270000,0.00,0.00,0.00,0.00,4449.10,3825.00,3825.00,80519.04,82773.32,82773.32,0.00,0,0.00,0.00,2.00,Active,7,0,7.69
6,1079071,171320,-6,353451.65,585000,67500.00,67500.00,0.00,0.00,14684.17,15750.00,15750.00,345433.86,351881.15,351881.15,1.00,1,0.00,0.00,6.00,Active,0,0,0.00
7,2095912,118650,-7,47962.12,45000,45000.00,45000.00,0.00,0.00,0.00,264.69,0.00,44735.31,47962.12,47962.12,1.00,1,0.00,0.00,51.00,Active,0,0,2.08
8,2181852,367360,-4,291543.08,292500,90000.00,289339.42,0.00,199339.42,130.50,4093.51,4093.51,285376.41,286831.58,286831.58,3.00,8,0.00,5.00,3.00,Active,0,0,0.00
9,1235299,203885,-5,201261.20,225000,76500.00,111026.70,0.00,34526.70,6338.34,45000.00,45000.00,192793.27,197224.70,197224.70,3.00,9,0.00,6.00,38.00,Active,0,0,6.25


In [17]:
CCB.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3840312 entries, 0 to 3840311
Data columns (total 27 columns):
 #   Column                      Dtype  
---  ------                      -----  
 0   SK_ID_PREV                  int64  
 1   SK_ID_CURR                  int64  
 2   MONTHS_BALANCE              int64  
 3   AMT_BALANCE                 float64
 4   AMT_CREDIT_LIMIT_ACTUAL     int64  
 5   AMT_DRAWINGS_ATM_CURRENT    float64
 6   AMT_DRAWINGS_CURRENT        float64
 7   AMT_DRAWINGS_OTHER_CURRENT  float64
 8   AMT_DRAWINGS_POS_CURRENT    float64
 9   AMT_INST_MIN_REGULARITY     float64
 10  AMT_PAYMENT_CURRENT         float64
 11  AMT_PAYMENT_TOTAL_CURRENT   float64
 12  AMT_RECEIVABLE_PRINCIPAL    float64
 13  AMT_RECIVABLE               float64
 14  AMT_TOTAL_RECEIVABLE        float64
 15  CNT_DRAWINGS_ATM_CURRENT    float64
 16  CNT_DRAWINGS_CURRENT        int64  
 17  CNT_DRAWINGS_OTHER_CURRENT  float64
 18  CNT_DRAWINGS_POS_CURRENT    float64
 19  CNT_INSTALMENT_MATURE

In [41]:
CCB_FEATURES=CCB[['SK_ID_CURR','NO_LOANS','DPD_COUNT','AVG_DPD','PERCENTAGE_MISSED_PAYMENTS']]
CCB_FEATURES.head()

,SK_ID_CURR,NO_LOANS,DPD_COUNT,AVG_DPD,PERCENTAGE_MISSED_PAYMENTS
0,378907,1,1.00,0.13,2.13
1,363914,1,1.00,0.01,1.04
2,371185,1,0.00,0.00,0.00
3,337855,1,0.00,0.00,6.67
4,126868,1,1.00,0.01,1.04


In [42]:
CCB_FEATURES=CCB_FEATURES.set_index('SK_ID_CURR')

In [43]:
CCB_FEATURES.head()

,NO_LOANS,DPD_COUNT,AVG_DPD,PERCENTAGE_MISSED_PAYMENTS
SK_ID_CURR,,,,
378907,1,1.00,0.13,2.13
363914,1,1.00,0.01,1.04
371185,1,0.00,0.00,0.00
337855,1,0.00,0.00,6.67
126868,1,1.00,0.01,1.04


In [44]:
CCB_FEATURES=CCB_FEATURES.drop_duplicates()

In [47]:
CCB_FEATURES.shape

(8240, 4)